In [9]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import math

In [2]:
def recommender(positive_list = None, negative_list = None, topn = 5):
    model = Word2Vec.load("item2Vec_20200328")
    recommend_anime_ls = ["241"]
    for animeId, prob in model.wv.most_similar_cosmul(positive = positive_list, negative = negative_list, topn = topn):
        recommend_anime_ls.append(animeId)

    return recommend_anime_ls

In [5]:
anime = pd.read_csv("~/Data/anime.csv")
ls = recommender(positive_list = ["241"], topn = 5)
print(anime[anime["anime_id"].isin(ls)])

      anime_id                        name  \
2508        53               Ai Yori Aoshi   
3216       196                Onegai☆Twins   
3382       193                    Maburaho   
3728       487  Girls Bravo: Second Season   
4105        63                       DearS   
4551       241   Girls Bravo: First Season   

                                                  genre type episodes  rating  \
2508       Comedy, Drama, Harem, Romance, Slice of Life   TV       24    7.24   
3216               Comedy, Drama, Harem, School, Sci-Fi   TV       12    7.05   
3382  Comedy, Drama, Ecchi, Harem, Magic, Romance, S...   TV       24    7.01   
3728     Comedy, Ecchi, Fantasy, Harem, Romance, School   TV       13    6.90   
4105              Comedy, Ecchi, Harem, Romance, Sci-Fi   TV       12    6.80   
4551     Comedy, Ecchi, Fantasy, Harem, Romance, School   TV       11    6.69   

      members  
2508    66597  
3216    59164  
3382    61004  
3728    57110  
4105    91037  
4551    84395

In [7]:
df = pd.read_csv("~/Data/clean_rating4.csv")
train_ratio = 0.9
test_data = df[int(len(df) * train_ratio):]

In [5]:
def ndcg(k, ranklist, testlist):
    idcg_k, dcg_k = 0, 0
    if len(testlist) < k:
        k = len(testlist)
    for i in range(k):
        idcg_k += 1 / math.log(i + 2, 2)
        
    s = set(testlist)
    hits = [idx for idx, val in enumerate(ranklist) if val in s]
    count = len(hits)
    
    for i in range(count):
        dcg_k += 1 / math.log(hits[i] + 2, 2)
        
    return float(dcg_k / idcg_k)

In [10]:
users = list(set(test_data.user_id.tolist()))
final = []
for k in range(10):
    precision, ndcg_k = [], []
    for user in users:
        target_items = [data[2] for data in test_data[test_data.user_id == user].values if data[3] == 10]
        if not target_items: 
            for i in range(9, -1, -1):
                target_items = [data[2] for data in test_data[test_data.user_id == user].values if data[3] == i]
                if target_items:
                    target_item = target_items[0]
                    break
        else:
            target_item = target_items[0]
            
        rec = list(map(int, recommender(positive_list = [str(target_item)], topn = 5)[1:]))

        cur_items = [data[2] for data in test_data[test_data.user_id == user].values if data[3] > k]
        precision.append(len(set(rec) & set(cur_items)) / len(rec))
        ndcg_k.append(ndcg(10, rec, cur_items))
        
    print(np.mean(ndcg_k))
    final.append(np.mean(ndcg_k))

0.41440766003383633
0.4142006985395797
0.4132643670788073
0.413078465075223
0.41151754066697654
0.40885232930689897
0.3986637612659749
0.35710539174500555
0.2558028601985228


ZeroDivisionError: division by zero

In [21]:
print(final)

[0.43611111111111117, 0.43611111111111117, 0.43472222222222223, 0.4305555555555556, 0.4166666666666667, 0.3819444444444444, 0.30972222222222223, 0.18981481481481483, 0.09675925925925927, 0.033796296296296297]
